# naver blog 크롤링

In [1]:
from urllib.parse import quote_plus
import requests
import lxml.html

In [47]:
search_url = ('http://section.blog.naver.com/sub/SearchBlog.nhn?type=post&option.keyword={query}'
              '&option.page.currentPage={page}&option.orderBy=sim')

query = quote_plus('딥러닝 바이오')

In [48]:
url = search_url.format(query=query, page=2)
res = requests.get(url)
root = lxml.html.fromstring(res.text)

In [49]:
for link in root.cssselect('h5 a'):
    print(link.text_content(), link.attrib['href'])

NEC, 귀구멍 생체인식 기술과 딥러닝으로 실내 대상자의 '위치측정기술' 개발 http://blog.naver.com/seminartoday?Redirect=Log&logNo=220850742260&from=section
차세대 투자처 전기차배터리ᆞ딥러닝 ...미리 선점 돈 벌자. http://blog.naver.com/ksmarkman1?Redirect=Log&logNo=220782402770&from=section
[바이오코리아] BIO KOREA 2016 현장 스케치 - 전시회 및 컨퍼런스 http://lse7927.blog.me/220672847984
삼성메디슨, 세계최초 '딥러닝' 기술접목 유방암 진단기 개발 http://blog.naver.com/dhseo1308?Redirect=Log&logNo=220719552807&from=section
딥러닝과 빅데이터 산업동향 http://itdcenter.blog.me/220762637181
제4차 산업혁명? 세상을 바꿀 테크놀로지 100 http://gami1023.blog.me/220976312058
2017년 상반기 GSAT 대비 시사 정리 – 5. 3월 중순~4월 초 추가분 http://blog.naver.com/yankmo?Redirect=Log&logNo=220974707792&from=section
[IT 위클리] 액션 스타 이연걸, 창업 대열에 합류 http://blog.naver.com/china_lab?Redirect=Log&logNo=220943673693&from=section
[중국의 반격]중국의 인공지능 어디까지 왔나? http://blog.naver.com/china_lab?Redirect=Log&logNo=220967572423&from=section
AI-바이오 융합이 가져올 경제적 효과 http://hyungrac1000.blog.me/220916319450


## URL 분해

In [50]:
from urllib.parse import parse_qs, urlparse

In [51]:
result = urlparse('http://blog.naver.com/dhseo1308?Redirect=Log&logNo=220719552807&from=section')
result

ParseResult(scheme='http', netloc='blog.naver.com', path='/dhseo1308', params='', query='Redirect=Log&logNo=220719552807&from=section', fragment='')

In [52]:
# path : user id, logNo : blog post id

In [53]:
result.path

'/dhseo1308'

In [54]:
qs = parse_qs(result.query)
qs

{'Redirect': ['Log'], 'from': ['section'], 'logNo': ['220719552807']}

In [55]:
qs['logNo'][0]

'220719552807'

In [56]:
# 블로그 해당글의 최종 URL
post_url = 'http://blog.naver.com/PostView.nhn?blogId={}&logNo={}'.format(result.path[1:], qs['logNo'][0])
post_url

'http://blog.naver.com/PostView.nhn?blogId=dhseo1308&logNo=220719552807'

## 게시물 내용 가져오기

In [57]:
post_res = requests.get(post_url)
post_root = lxml.html.fromstring(post_res.text)

In [58]:
# css : class는 "."으로, id 는 "#" 으로 구분

In [59]:
post_root.cssselect('div#postViewArea')[0].text_content()

"\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t \r\n삼성메디슨, 세계최초 '딥러닝' 기술접목 유방암 진단기 개발\r\nhttp://www.edaily.co.kr/news/NewsRead.edy?SCD=JC51&newsid=01708886612652856&DCD=A00305&OutLnkChk=Y\r\n\xa0\r\n솔고바이오(043100) 삼성메디슨 자회사 지분보유[특징주]솔고바이오, 삼성메디슨 등 美·中에 초음파 진단기 수출…↑http://view.asiae.co.kr/news/view.htm?idxno=2016020313405605593\r\n솔고바이오는 삼성메디슨 계열사 메디너스의 지분 14.02%를 보유한 것으로 알려져 관련주로 부각됐다.\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t"

## 블로그 크롤링

In [60]:
import tqdm
from urllib.parse import urljoin

In [61]:
keyword = '딥러닝 바이오'
query = quote_plus(keyword)

In [62]:
search_url = ('http://section.blog.naver.com/sub/SearchBlog.nhn?type=post&option.keyword={query}'
              '&option.page.currentPage={page}&option.orderBy=sim')

In [66]:
posts = []
for page in tqdm.tqdm_notebook(range(1, 11)):
    url = search_url.format(query=query, page=page)
    res = requests.get(url)
    root = lxml.html.fromstring(res.text)
    
    for link in root.cssselect('h5 a'):
        link_url = link.attrib['href']

        # 다른 형식의 주소는 무시
        if not link_url.startswith('http://blog.naver.com'):
            continue

        # 진짜 주소
        result = urlparse(link_url)
        blog_id = result.path[1:]
        qs = parse_qs(result.query)
        post_id = qs['logNo'][0]
        post_url = 'http://blog.naver.com/PostView.nhn?blogId={}&logNo={}'.format(blog_id, post_id)
        
        # 본문 가져오기
        post_res = requests.get(post_url)
        post_root = lxml.html.fromstring(post_res.text)
        
        try:
            body = post_root.cssselect('div#postViewArea')[0]
            posts.append(body.text_content())
        except IndexError:
            continue

In [67]:
len(posts)

55

In [68]:
# CSV로 저장

In [69]:
import csv
import re

In [70]:
with open('posts.csv', 'w', encoding='utf8', newline='') as f:
    w = csv.writer(f)
    for post in posts:
        post_short = re.sub(r'\s+', ' ', post)  # 모든 종류의 공백을 빈 칸 하나로 바꿈 (엑셀에서 보기 좋게)
        w.writerow([post_short])

In [71]:
# csv에서 불러오기

In [88]:
posts = []
with open('posts.csv', encoding='utf8') as f:
    reader = csv.reader(f)
    for row in reader:
        post = row[0]
        if len(post) > 100:
            posts.append(post)

In [89]:
len(posts)

55

In [90]:
posts[:1]

[' 전문가처럼 정확한 인공지능 혈압측정기술 최초 개발- 뇌신경 모방 딥러닝 기술 적용, 바이오진단용으로 활용 기대 등록일 2016-11-09 전문가처럼 정확한 인공지능 혈압측정기술 최초 개발 - 뇌신경 모방 딥러닝 기술 적용, 바이오진단용으로 활용 기대 - □ 미래창조과학부(장관 최양희)는 인공지능 딥러닝 기술을 이용하여 의사 등 전문가가 측정하는 수준의 정확성을 갖는 혈압측정기술을 한양대 장준혁 교수 연구팀이 세계 최초로 개발하였다고 밝혔다. * 인공지능 딥러닝 기술 : 대용량의 데이터나 복잡한 자료들 속에서 사람의 뇌신경에서 학습하는 메커니즘처럼 컴퓨터를 학습시켜 핵심적인 내용과 기능을 분류하거나 군집화할 수 있는 기계학습방법 o 혈압은 사람의 건강상태를 보여주는 가장 중요한 생체신호이다. 혈압을 잴 때는 현재, 압박대를 감아 재는 간접적인 방식이 사용되는데, 전문가가 청진기로 재는 것처럼 정확도가 높은 혈압측정 기술이 개발되었다. □ 장준혁 교수 연구팀(한양대)은 미래창조과학부 기초연구사업(개인연구)의 지원으로 연구를 수행했으며, 이 연구는 국제적인 학술지 IEEE 산업정보 트랜잭션 10월 15일자에 게재되었다. o 논문명과 저자 정보는 다음과 같다. - 논문명 : Oscillometric Blood Pressure Estimation Based on Deep Learning - 저자 정보 : 장준혁 교수(교신저자, 한양대), 이수정(제1저자, 한양대) □ 논문의 주요 내용은 다음과 같다. 1. 연구의 필요성 ○ 혈압은 환자의 건강상태를 보여주는 가장 중요한 생체신호이다. 2012년 세계보건기구 (WHO) 발표기준 전 세계 사망자 가운데 31%가 심혈관질환 사망자로 발표되었다. 현재 오실로메트릭 혈압측정방법이 수축기 혈압과 이완기 혈압을 추정하는 방법으로 널리 사용되고 있지만 경험적인 방법(최대진폭방법)이라는 한계가 있어 수학적 방법, 뉴럴 네트워크 방법* 등 여러 방법들이 대안으로 시도되고 있지만 만족할 만한 연구 성과를 얻을 수 없었다. * 뉴럴 네

## TDM 만들기

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Komoran

In [92]:
tagger = Komoran()   # 형태소 분석기

In [93]:
def get_noun(text):
    nouns = tagger.nouns(text)
    return [n for n in nouns if len(n) > 1]  # 2글자 이상인 명사만 추출

In [94]:
cv = CountVectorizer(tokenizer=tagger.nouns, max_features=500)

In [95]:
tdm = cv.fit_transform(posts)

In [96]:
import numpy

In [97]:
numpy.savez('blog_tdm.npz', tdm)   # TDM 저장

## 단어 목록 저장

In [98]:
import json

In [99]:
with open('blog_words.json', 'w', encoding='utf8') as f:
    json.dump(cv.get_feature_names(), f)

## TDM 불러오기

In [100]:
data = numpy.load('blog_tdm.npz', encoding='latin1')  # encoding은 python 2에서 만든 것을 3에서 불러올 때만 사용

In [101]:
data.files

['arr_0']

In [103]:
tdm = data['arr_0'].item()
tdm

<55x500 sparse matrix of type '<class 'numpy.int64'>'
	with 7573 stored elements in Compressed Sparse Row format>

## 단어 목록 불러오기

In [104]:
import json

In [105]:
with open('blog_words.json', encoding='utf8') as f:
    words = json.load(f)

In [122]:
words[:30]

['4차 산업',
 '가격',
 '가정',
 '가지',
 '가치',
 '가트너',
 '간',
 '감소',
 '강연',
 '강화',
 '개',
 '개념',
 '개발',
 '개별',
 '개선',
 '개월',
 '개인',
 '개최',
 '건',
 '건강',
 '검사',
 '것',
 '게임',
 '결과',
 '결정',
 '결합',
 '경기',
 '경우',
 '경쟁',
 '경쟁력']

## 차원 축소

In [108]:
from sklearn.decomposition import TruncatedSVD   # = PCA
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

In [109]:
svd = TruncatedSVD(n_components=10)   # 10 개의 차원으로 축소.
pos = svd.fit_transform(tdm)

In [110]:
pos.shape

(55, 10)

In [111]:
pos[0, :]

array([ 77.87838688, -70.59942751,  -3.16606568,  96.71254329,
       -32.95523998,  37.96075643, -16.19291974,  -2.48603469,
        -3.04703008,   3.478666  ])

## 정규화

In [124]:
# 글의 방향성만 남도록 길이를 동일하게 조정.

In [112]:
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)   # svd 와 normalizer를 순서대로 적용.
pos = lsa.fit_transform(tdm)

In [113]:
pos.shape

(55, 10)

In [114]:
pos[0, :]

array([ 0.51097153, -0.4632132 , -0.02077299,  0.63454521, -0.21622285,
        0.24906931, -0.10624609, -0.01631449, -0.01997289,  0.02283289])

In [115]:
sum(pos[0, :] ** 2)

1.0

## 클러스터링

In [123]:
# 각 문서간의 유사성을 기준으로 군집화

In [116]:
from sklearn.cluster import KMeans

In [117]:
km = KMeans(n_clusters=5)

In [118]:
km.fit(pos)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [119]:
km.labels_

array([4, 0, 1, 4, 3, 1, 3, 3, 3, 4, 2, 3, 1, 1, 2, 1, 3, 1, 3, 3, 1, 3, 4,
       0, 0, 0, 1, 0, 0, 2, 0, 1, 3, 0, 4, 2, 0, 2, 3, 2, 2, 1, 4, 1, 3, 3,
       2, 3, 1, 0, 2, 1, 1, 3, 3])